In [1]:
from nilearn import datasets, plotting
from nilearn.input_data import NiftiSpheresMasker
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn.masking import compute_epi_mask
from nilearn.masking import apply_mask

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

C:\Users\hp\AppData\Local\Programs\Python\Python37\lib\site-packages\nilearn\datasets\__init__.py:90: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


In [2]:
#adhd_dataset = datasets.fetch_adhd(n_subjects=40)
atlas = datasets.fetch_atlas_msdl()
atlas_filename = atlas['maps']

from nilearn.input_data import NiftiMapsMasker
masker = NiftiMapsMasker(maps_img=atlas_filename, standardize=True,
                         memory='nilearn_cache', verbose=5)

C:\Users\hp\AppData\Local\Programs\Python\Python37\lib\site-packages\numpy\lib\npyio.py:2349: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  output = genfromtxt(fname, **kwargs)


In [3]:
import pandas as pd
df = pd.read_csv('patients.csv',usecols =['Subject','Labels'])
df = df[26:26+94]

In [4]:
from nilearn.connectome import ConnectivityMeasure
func_filename =str('F:\\Data Analytics\\BTP I\\Data\\www.nitrc.org\\ir\\') + str(df.values[1,0])+str('_1\\rest_1\\NIfTI\\rest.nii.gz')
time_series = masker.fit_transform(func_filename)
print(time_series.shape)
correlation_measure = ConnectivityMeasure(kind='correlation')
correlation_matrix = correlation_measure.fit_transform([time_series])[0]
print(correlation_matrix.shape)

[NiftiMapsMasker.fit_transform] loading regions from C:\Users\hp\nilearn_data\msdl_atlas\MSDL_rois\msdl_rois.nii
Resampling maps
[Memory]0.1s, 0.0min    : Loading resample_img...
C:\Users\hp\AppData\Local\Programs\Python\Python37\lib\site-packages\nilearn\_utils\cache_mixin.py:303: UserWarning: memory_level is currently set to 0 but a Memory object has been provided. Setting memory_level to 1.
  warnings.warn("memory_level is currently set to 0 but "
________________________________________resample_img cache loaded - 1.4s, 0.0min
________________________________________________________________________________
[Memory] Calling nilearn.input_data.base_masker.filter_and_extract...
filter_and_extract(('F:\\Data Analytics\\BTP '
 'I\\Data\\www.nitrc.org\\ir\\KKI_1019436_1\\rest_1\\NIfTI\\rest.nii.gz'), 
{ 'allow_overlap': True,
  'detrend': False,
  'dtype': None,
  'high_pass': None,
  'high_variance_confounds': False,
  'low_pass': None,
  'maps_img': 'C:\\Users\\hp/nilearn_data\\msdl_atl

In [5]:
from nilearn.connectome import ConnectivityMeasure
corr_data = []
#variance = []
print('|'*40)
for i in range(94):
    func_filename =str('F:\\Data Analytics\\BTP I\\Data\\www.nitrc.org\\ir\\') + str(df.values[i,0])+str('_1\\rest_1\\NIfTI\\rest.nii.gz')
    time_series = masker.fit_transform(func_filename)
    correlation_measure = ConnectivityMeasure(kind='correlation')
    correlation_matrix = correlation_measure.fit_transform([time_series])[0]
    corr_data.append(correlation_matrix)
    #pca = PCA(n_components=10)
    #data = pca.fit_transform(time_series[:75,:])
    #exp_variance = pca.explained_variance_ratio_
    #pca_data.append(data)
    #variance.append(exp_variance)
    #print('|',end='')

an fix the problem.
  verbose=self.verbose)
_______________________________________________filter_and_extract - 5.3s, 0.1min
[NiftiMapsMasker.fit_transform] loading regions from C:\Users\hp\nilearn_data\msdl_atlas\MSDL_rois\msdl_rois.nii
[Memory]133.9s, 2.2min  : Loading filter_and_extract...
__________________________________filter_and_extract cache loaded - 0.0s, 0.0min
[NiftiMapsMasker.fit_transform] loading regions from C:\Users\hp\nilearn_data\msdl_atlas\MSDL_rois\msdl_rois.nii
Resampling maps
[Memory]135.0s, 2.3min  : Loading resample_img...
________________________________________resample_img cache loaded - 1.4s, 0.0min
[Memory]136.9s, 2.3min  : Loading filter_and_extract...
__________________________________filter_and_extract cache loaded - 0.0s, 0.0min
[NiftiMapsMasker.fit_transform] loading regions from C:\Users\hp\nilearn_data\msdl_atlas\MSDL_rois\msdl_rois.nii
Resampling maps
[Memory]138.0s, 2.3min  : Loading resample_img...
________________________________________resample_

In [9]:
################## SAVING DATA ######################
import pickle
with open("Data/kki_corr_msdl.txt", "wb") as fp: 
    pickle.dump(corr_data, fp)

In [28]:
################### LOADING DATA ######################
import pickle
with open("Data/kki_corr_msdl.txt", "rb") as fp:
    corr_data = pickle.load(fp)

In [12]:
corr_data[93].shape

(39, 39)

In [29]:
for i in range(len(corr_data)):
    corr_data[i] = corr_data[i].reshape(39,39,1) #Time, PCA, nsample
    
corr_data[90].shape

(39, 39, 1)

In [30]:
train = np.array(corr_data[:80])
test = np.array(corr_data[80:])

y = df.values[:,1]
y_train = y[:80]
y_test = y[80:]

In [6]:
################## TESTING WITH NILEARN 40 ###########################
import pickle
with open("Data/nilearn_pca_msdl.txt", "rb") as fp:
    data = pickle.load(fp)
    
for i in range(len(data)):
    data[i] = data[i].reshape(75,10,1) #Time, PCA, nsample
test = np.array(data)
test=np.asarray(test).astype(np.float32)
df2 = pd.read_csv('nilearn.csv',usecols =['adhd'])
y_test = df2['adhd']

In [37]:
import tensorflow as tf
import keras

from tensorflow.keras import datasets, layers, models
from keras.layers import Dropout

model = models.Sequential()
model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=(39, 39, 1))) #Time, PCA, nsample
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(8, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
#model.add(layers.Conv2D(32, (3, 3), activation='relu'))
#model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(200, activation='relu'))
#model.add(Dropout(0.2))
model.add(layers.Dense(50, activation='relu'))
#model.add(Dropout(0.2))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 37, 37, 16)        160       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 18, 18, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 16, 16, 8)         1160      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 8, 8, 8)           0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 200)               102600    
_________________________________________________________________
dense_10 (Dense)             (None, 50)               

In [38]:
#model.compile(loss=keras.losses.categorical_crossentropy,
#              optimizer=keras.optimizers.Adadelta(),
#              metrics=['accuracy'])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [39]:
train=np.asarray(train).astype(np.float32)
y_train=np.asarray(y_train).astype(np.int)

test=np.asarray(test).astype(np.float32)
y_test=np.asarray(y_test).astype(np.int)

In [40]:
batch_size = 5
num_epoch = 20
#model training
model_log = model.fit(train, y_train,
          batch_size=batch_size,
          epochs=num_epoch,
          verbose=1,validation_data=(test, y_test),) #class_weight={0:1, 1:3})

Epoch 1/20
16/16 [==============================] - 1s 18ms/step - loss: 0.6459 - accuracy: 0.6916 - val_loss: 0.5800 - val_accuracy: 0.8571
Epoch 2/20
16/16 [==============================] - 0s 7ms/step - loss: 0.6289 - accuracy: 0.6775 - val_loss: 0.4549 - val_accuracy: 0.8571
Epoch 3/20
16/16 [==============================] - 0s 7ms/step - loss: 0.5737 - accuracy: 0.7256 - val_loss: 0.5059 - val_accuracy: 0.8571
Epoch 4/20
16/16 [==============================] - 0s 7ms/step - loss: 0.6304 - accuracy: 0.6150 - val_loss: 0.4901 - val_accuracy: 0.8571
Epoch 5/20
16/16 [==============================] - 0s 9ms/step - loss: 0.5328 - accuracy: 0.7356 - val_loss: 0.6365 - val_accuracy: 0.7857
Epoch 6/20
16/16 [==============================] - 0s 8ms/step - loss: 0.5500 - accuracy: 0.7916 - val_loss: 0.5366 - val_accuracy: 0.8571
Epoch 7/20
16/16 [==============================] - 0s 7ms/step - loss: 0.4565 - accuracy: 0.7695 - val_loss: 0.6618 - val_accuracy: 0.7143
Epoch 8/20
16/16 [=

In [41]:
y_pred = model.predict_classes(train)
adhd = [0,0]
typic = [0,0]
for i in range(len(y_pred)):
    if(y_pred[i]==1 and y_train[i]==1):
        adhd[0] = adhd[0]+1
        adhd[1] = adhd[1]+1
    elif(y_pred[i]==0 and y_train[i]==1):
        adhd[1] = adhd[1]+1
    if(y_pred[i]==0 and y_train[i]==0):
        typic[0] = typic[0]+1
        typic[1] = typic[1]+1
    elif(y_pred[i]==1 and y_train[i]==0):
        typic[1] = typic[1]+1
    
print(adhd, typic)

[23, 23] [57, 57]


In [43]:
y_pred = model.predict(test)
for i in range()

array([[2.7972859e-01],
       [5.0720507e-01],
       [9.8586679e-03],
       [1.6008238e-05],
       [8.1600547e-03],
       [8.7103927e-01],
       [4.4779289e-01],
       [4.4251859e-02],
       [1.7471433e-02],
       [4.7726035e-03],
       [3.2961369e-04],
       [3.6011130e-01],
       [9.2913926e-01],
       [5.2858108e-05]], dtype=float32)

In [42]:
y_pred = model.predict_classes(test)
adhd = [0,0]
typic = [0,0]
for i in range(len(y_pred)):
    if(y_pred[i]==1 and y_test[i]==1):
        adhd[0] = adhd[0]+1
        adhd[1] = adhd[1]+1
    elif(y_pred[i]==0 and y_test[i]==1):
        adhd[1] = adhd[1]+1
    if(y_pred[i]==0 and y_test[i]==0):
        typic[0] = typic[0]+1
        typic[1] = typic[1]+1
    elif(y_pred[i]==1 and y_test[i]==0):
        typic[1] = typic[1]+1
    
print(adhd, typic)

[0, 2] [9, 12]


In [22]:
################## TESTING WITH NILEARN 40 ###########################
import pickle
with open("Data/nilearn_pca_msdl.txt", "rb") as fp:
    data = pickle.load(fp)
    
for i in range(len(data)):
    data[i] = data[i].reshape(75,10,1) #Time, PCA, nsample
test = np.array(data)
test=np.asarray(test).astype(np.float32)
df2 = pd.read_csv('nilearn.csv',usecols =['adhd'])
y_test = df2['adhd']